# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2023-spring@googlegroups.com

Slide:    [Link](https://docs.google.com/presentation/d/18LkR8qulwSbi3SVoLl1XNNGjQQ_qczs_35lrJWOmHCk/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/t/c76950cc460140eba30a576ca7668d28)

#colab code

# 結果說明

由於我是參攷著網路上的資源進行輔助網絡aux = Auxiliary（）.cuda（）的編寫，囙此許多操作是與resnet模型並行操作，包括計算損失率和生成最後結果等，囙此最後會生成兩個文件prediction_a.csv和prediction.csv。 我在所有輔助網絡模型的相關檔和程式都加了_a的尾碼。 但是，我最後提交的是prediction_a.csv文件作為最終結果，希望助教老師在驗證程式中也要使用prediction_a.csv文件進行驗證。

# 簡單來說，最後生成兩個結果，要提交prediction_a.csv



# Result Description

As I am referring to the resources on the network to write the auxiliary network aux=Auxiliary(). cuda(), many of these operations are parallel to the resnet model, including calculating loss rates and generating final results. As a result, two files (prediction_a.csv) and (prediction.csv) will be generated. I have added files and programs related to all auxiliary network models_ The tail code of a. However, the final result I submitted was the (prediction_a.csv) file, and I hope the teaching assistant will also use the (prediction_ a.csv) file for verification in the verification program.

# Simply put, to generate two final results, submit a (prediction_ a.csv) file

# # Reference link （all）
For specific location references, see the markdown and comments in the code below.

https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187


https://blog.csdn.net/weixin_46846685/article/details/127699597

http://www.taodudu.cc/news/show-4804642.html

https://blog.csdn.net/iwill323/article/details/127919157

https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187


# Set up the environment


## Package installation

In [1]:
# Training progress bar
!pip install -q qqdm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00


## Downloading data

In [2]:
!git clone https://github.com/chiyuanhsiao/ml2023spring-hw8

Cloning into 'ml2023spring-hw8'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), 1.69 KiB | 864.00 KiB/s, done.
Filtering content: 100% (2/2), 1.36 GiB | 50.38 MiB/s, done.


# Import packages

In [3]:
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [4]:

train = np.load('ml2023spring-hw8/trainingset.npy', allow_pickle=True)
test = np.load('ml2023spring-hw8/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [5]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(730)

# Autoencoder

# Models & loss

# reference link
# Change the network structure (increase the number of layers)
https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187

In [6]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(), 
            nn.Linear(256, 64), 
            nn.ReLU(), 
            nn.Linear(64, 10)
        )    # Hint: dimension of latent space can be adjusted
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers
        self.decoder = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(), 
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 64 * 64 * 3), 
            nn.Tanh()
            # reference link
            # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
            # increase the number of layers
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 24, 4, stride=2, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(24, 24, 4, stride=2, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 96, 4, stride=2, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(96*4*4, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 10),
            nn.BatchNorm1d(10),
            nn.ReLU(),
        )   # Hint:  dimension of latent space can be adjusted
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers

        self.decoder = nn.Sequential(
			      nn.Linear(10, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 96*4*4),
            nn.BatchNorm1d(96*4*4),
            nn.ReLU(),
            nn.Unflatten(1, (96, 4, 4)),
            nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Flatten()
            # reference link
            # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
            # increase the number of layers

        )
        self.enc_out_1 = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(64*4*4, 128),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            
            nn.Dropout(0.2),
            nn.Linear(128, 10),
            #nn.BatchNorm1d(10),
            nn.ReLU()
        )
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers

        self.enc_out_2 = nn.Sequential(
            #nn.Conv2d(24, 48, 4, stride=2, padding=1),
            #nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64*4*4, 128),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
            
            nn.Dropout(0.2),
            nn.Linear(128, 10),
            #nn.BatchNorm1d(10),
            nn.ReLU()
        )
        # Hint: can add more layers to encoder and decoder
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers

        self.decoder = nn.Sequential(
			      nn.Linear(10, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 32*4*4),
            nn.BatchNorm1d(32*4*4),
            nn.ReLU(),
            nn.Unflatten(1, (32, 4, 4)),
            nn.ConvTranspose2d(32, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        # reference link
        # https://blog.csdn.net/qq_43613342/article/details/127018677?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-127018677-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        # increase the number of layers
        

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# reference link
# Decoder auxiliary network
https://blog.csdn.net/weixin_46846685/article/details/127699597

https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187

In [7]:
class ensemble(nn.Module):
    def __init__(self):
        super().__init__()
        # reference link
        # Decoder auxiliary network
        # https://blog.csdn.net/weixin_46846685/article/details/127699597
        self.encoder_1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 64 * 3, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(), 
            nn.Dropout(0.2),
            nn.Linear(256, 64), 
            nn.BatchNorm1d(64),
            nn.ReLU(), 
        )
        # reference link
        # Decoder auxiliary network
        # https://blog.csdn.net/weixin_46846685/article/details/127699597
        self.encoder_2 = nn.Sequential(
            nn.Conv2d(3, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(128*4*4, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )
        # reference link
        # Decoder auxiliary network
        # https://blog.csdn.net/weixin_46846685/article/details/127699597
        self.decoder = nn.Sequential(
                nn.Linear(192, 128*4*4),
                nn.BatchNorm1d(128*4*4),
                nn.ReLU(),
                nn.Unflatten(1, (128, 4, 4)),
                nn.ConvTranspose2d(128, 256, 4, stride=2, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 256, 4, stride=2, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
                nn.Tanh(),
        )
        
    def encoder(self, x):
        x1 = self.encoder_1(x)
        x2 = self.encoder_2(x)
        return torch.cat((x1, x2), 1)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# resnet network
# reference link
http://www.taodudu.cc/news/show-4804642.html



# Residual network
# reference link
https://blog.csdn.net/iwill323/article/details/127919157

In [8]:
class Residual_Block(nn.Module):
  # Residual network
  # reference link
  # https://blog.csdn.net/iwill323/article/details/127919157
    def __init__(self, ic, oc, stride=1):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(ic, oc, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(oc),
            nn.ReLU(inplace=True)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(oc, oc, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(oc),
        )
        
        self.relu = nn.ReLU(inplace=True)
    
        self.downsample = None
        if stride != 1 or (ic != oc):
            self.downsample = nn.Sequential(
                nn.Conv2d(ic, oc, kernel_size=1, stride=stride),
                nn.BatchNorm2d(oc),
            )
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample:
            residual = self.downsample(x)
            
        out += residual
        return self.relu(out)

## reference link
# http://www.taodudu.cc/news/show-4804642.html    
class ResNet(nn.Module):
    def __init__(self, block=Residual_Block, num_layers=[2, 1, 1, 1]):
        super().__init__()
        self.preconv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.layer0 = self.make_residual(block, 32, 64,  num_layers[0], stride=2)
        self.layer1 = self.make_residual(block, 64, 128, num_layers[1], stride=2)
        self.layer2 = self.make_residual(block, 128, 128, num_layers[2], stride=2)
        self.layer3 = self.make_residual(block, 128, 64, num_layers[3], stride=2)
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(64*4*4, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(64, 64*4*4),
            nn.BatchNorm1d(64*4*4),
            nn.ReLU(),
            nn.Unflatten(1, (64, 4, 4)),
            nn.ConvTranspose2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
    def make_residual(self, block, ic, oc, num_layer, stride=1):
        layers = []
        layers.append(block(ic, oc, stride))
        for i in range(1, num_layer):
            layers.append(block(oc, oc))
        return nn.Sequential(*layers)
    
    def encoder(self, x):
        x = self.preconv(x)
        x = self.layer0(x) #64*64 --> 32*32
        x = self.layer1(x) #32*32 --> 16*16
        x = self.layer2(x) #16*16 --> 8*8
        x = self.layer3(x) #8*8 --> 4*4
        x = self.fc(x)
        return x
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
# reference link
# Decoder auxiliary network
# https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187    
class Auxiliary(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.Linear(64, 64*4*4),
            nn.BatchNorm1d(64*4*4),
            nn.ReLU(),
            nn.Unflatten(1, (64, 4, 4)),
            nn.ConvTranspose2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )
        
    def forward(self, x):
        return self.decoder(x)

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [9]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


# resnet network
# reference link
http://www.taodudu.cc/news/show-4804642.html


# Parameter settings
# reference link
https://blog.csdn.net/iwill323/article/details/127919157

In [10]:
# Training hyperparameters
num_epochs = 70
batch_size = 128 # Hint: batch size may be lower
learning_rate = 1e-3

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

# Model

#model_type = 'vae'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_type = 'resnet'
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE(), 'ensemble':ensemble(), 'resnet':ResNet()}
model = model_classes[model_type].cuda()

# resnet network
# reference link
# http://www.taodudu.cc/news/show-4804642.html

aux = Auxiliary().cuda()
# reference link
# Decoder auxiliary network
# https://blog.csdn.net/weixin_46846685/article/details/127699597

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer_a = torch.optim.AdamW(aux.parameters(), lr=learning_rate)

## Training loop

# format writting
# reference link
https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187

In [ ]:

best_loss = np.inf
#model.train()
qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'), dynamic_ncols=False)
#qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()

    tot_loss_a = list()
    aux.train()
    temperature = epoch // 2 + 1

    for data in train_dataloader:

        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)

        # ===================forward=====================
        model.train()
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)
        loss = loss.mean()
        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # format writting
        # reference link
        # https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
        model.eval()
        z = model.encoder(img).detach_()
        output = output.detach_()
        output_a = aux(z)
        loss_a = (criterion(output_a, output).mul(temperature).exp())*criterion(output_a, img)
        loss_a = loss_a.mean()
        tot_loss_a.append(loss_a.item())
        optimizer_a.zero_grad()
        loss_a.backward()
        optimizer_a.step()

    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))

        torch.save(aux, 'best_model_a_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',

        'loss_a':f'{np.mean(tot_loss_a):.4f}'
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))
    # format writting
    # reference link
    # https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
    torch.save(aux, 'last_model_a_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                               
 0/70           -             -                                                 
Description   0.0% |                                       |                     Iters    Elapsed Time      Speed    epoch   loss   loss_a                      
 1/70   00:02:26<02:48:21  0.01it/s  1/70   0.1212  0.1573                      
Description   1.4% |                                       |                     Iters    Elapsed Time      Speed    epoch   loss   loss_a                      
 2/70   00:04:48<02:43:41  0.01it/s  2/70   0.0670  0.0658                      
Description   2.9% |█                                      |                     Iters    Elapsed Time      Speed    epoch   loss   loss_a                      
 3/70   00:07:11<02:40:33  0.01it/s  3/70   0.0601  0.0584                      
Description   4.3% |█                                      |                     Iters    Elapsed Time      Spee

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

# format writting
# reference link
https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187

In [ ]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
checkpoint_path_a = f'best_model_a_{model_type}.pt'
model = torch.load(checkpoint_path).cuda()
aux = torch.load(checkpoint_path_a).cuda()
model.eval()

# prediction file 
out_file = 'prediction.csv'

# format writting
# reference link
# https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
out_file_a = 'prediction_a.csv'

# format writting
# reference link
https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187

In [ ]:
anomality = list()
auxs = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):
    img = data.float().cuda()
    if model_type in ['fcn']:
      img = img.view(img.shape[0], -1)
    
    z = model.encoder(img)

    output = model(img)
    output_a = aux(z)
    if model_type in ['vae']:
      output = output[0]
    if model_type in ['fcn']:
        loss = eval_loss(output, img).mean(-1)
        #loss = eval_loss(output, img).sum(-1)
    else:
        #loss = eval_loss(output, img).sum([1, 2, 3])
        #loss_a = eval_loss(output_a, img).sum([1, 2, 3])
        loss = eval_loss(output, img).mean([1, 2, 3])
        loss_a = eval_loss(output_a, img).mean([1, 2, 3])
    anomality.append(loss)
    auxs.append(loss_a)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

# format writting
# reference link
# https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
auxs = torch.cat(auxs, axis=0)
auxs = torch.sqrt(auxs).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')

# format writting
# reference link
# https://blog.csdn.net/weixin_42369818/article/details/125292835?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168321094516800182172074%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168321094516800182172074&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-125292835-null-null.142^v86^insert_down1,239^v2^insert_chatgpt&utm_term=%E6%9D%8E%E5%AE%8F%E6%AF%85hw8&spm=1018.2226.3001.4187
df_a = pd.DataFrame(auxs, columns=['score'])
df_a.to_csv(out_file_a, index_label = 'ID')